# Lesson 3：Python 魔术方法入门

本节课将介绍 Python 中的魔术方法（magic methods），说明它们解决的问题，并通过示例演示常见的使用模式。理解这些方法能帮助我们定制对象的行为，让类与内置语法特性（如运算、比较、上下文管理等）自然配合。

## 什么是魔术方法？

魔术方法是以双下划线开头和结尾的特殊方法，例如 `__init__`、`__len__`、`__str__`。当对象与特定语法或内置函数交互时，Python 会自动调用这些方法，使得自定义类可以表现得像内置类型一样自然。

---

## 一、Python 的核心哲学：一切皆对象

在 Python 里，**几乎所有东西都是对象**——数字、字符串、函数、模块、类、文件、迭代器……

> 每个对象都遵循一套通用协议（protocol），比如：
>
> * 能被 `len()` 调用（实现 `__len__`）
> * 能被 `for` 循环（实现 `__iter__`/`__next__`）
> * 能被 `+`、`in`、`==` 等操作符使用

**魔术方法就是这些协议的实现入口。**

---

## 二、为什么需要它：让对象像内建类型一样“自然”

### ✅ 1. 让你的类更 Pythonic（像内建类型）

假设我们实现一个向量类：

```python
v1 + v2      # 想写成这样，而不是 v1.add(v2)
len(v1)      # 想要计算维度，而不是 v1.length()
v1 == v2     # 比较是否相等
```

如果不用魔术方法，这些操作都需要手动定义普通方法；
但有了 `__add__`, `__len__`, `__eq__` 等，就能让类与内建类型无缝衔接。

---

### ✅ 2. 统一接口：所有对象都遵循相同协议

Python 内部操作符、内建函数（`len()`, `str()`, `bool()`, `sum()`）
其实都在背后调用对应的魔术方法。

| 表达式           | 实际调用                          |
| ------------- | ----------------------------- |
| `len(x)`      | `x.__len__()`                 |
| `x + y`       | `x.__add__(y)`                |
| `x in y`      | `y.__contains__(x)`           |
| `for a in b:` | `b.__iter__()` + `__next__()` |
| `bool(x)`     | `x.__bool__()`                |

> 有了统一协议，Python 就能用一种语法处理不同类型的对象（鸭子类型思想）。

---

### ✅ 3. 自定义行为：重载运算、访问控制、资源管理

魔术方法提供了“钩子（hook）”机制，让你能插手 Python 的默认行为。

| 场景    | 对应魔术方法                         | 作用            |
| ----- | ------------------------------ | ------------- |
| 对象创建  | `__new__`, `__init__`          | 控制实例化流程       |
| 运算重载  | `__add__`, `__mul__`, `__eq__` | 改写算术/逻辑运算     |
| 上下文管理 | `__enter__`, `__exit__`        | `with` 自动释放资源 |
| 属性访问  | `__getattr__`, `__setattr__`   | 拦截、验证、动态加载属性  |
| 函数式对象 | `__call__`                     | 让实例像函数一样被调用   |

---

### ✅ 4. 提高可读性与复用性

例如：

```python
with Database() as db:
    db.query("SELECT * FROM users")
```

背后就是利用了 `__enter__` / `__exit__`，从而统一资源管理风格（类似文件操作）。

再比如：

```python
user in group  # group.__contains__(user)
```

比 `group.has_user(user)` 语义更自然。

---

### ✅ 5. Python 内部机制依赖它

解释器在执行很多语法结构时会自动触发这些方法。
**如果你实现了它，就能改变解释器行为。**

举例：

```python
class A:
    def __getitem__(self, i):
        print("get", i)
        return i*2

a = A()
a[3]        # get 3
for x in a: # 调用 __iter__ 或 __getitem__ 迭代
    ...
```

这就是“语法糖背后的钩子机制”。

---

## 三、总结：为什么要学、为什么要用

| 目的                  | 收益                            |
| ------------------- | ----------------------------- |
| ✅ 让对象自然融入 Python 世界 | 能被 `len()`, `in`, `for` 等直接使用 |
| ✅ 增强代码可读性           | 操作更贴近直觉，不用记忆专有方法名             |
| ✅ 代码可复用性强           | 统一协议使各种工具（排序、序列化等）自动支持        |
| ✅ 掌握底层机制            | 理解 Python 的运行模型（解释器如何调用方法）    |
| ✅ 为高级特性打基础          | 自定义容器、ORM、框架、DSL 全靠这些机制       |

---

## 常见魔术方法分类

- **对象生命周期**：`__new__`、`__init__`、`__del__`
- **字符串表示**：`__repr__`、`__str__`
- **容器协议**：`__len__`、`__iter__`、`__getitem__`、`__contains__`
- **可调用对象**：`__call__`
- **数值运算与比较**：`__add__`、`__sub__`、`__lt__`、`__eq__` 等
- **上下文管理器**：`__enter__`、`__exit__`

接下来我们通过示例一步步演练这些方法的作用与写法。

### 1. 字符串表示：`__repr__` 与 `__str__`

- `__repr__`：面向开发者的调试表示，应尽量精确且可复现对象状态。
- `__str__`：面向用户的友好字符串，常用于打印或日志。

In [10]:
# 用于演示 __repr__ 和 __str__ 差异的二维向量类
class Vector2D:
    def __init__(self, x, y):
        # 保存水平分量
        self.x = x
        # 保存垂直分量
        self.y = y

    def __repr__(self):
        # 返回供开发者调试的字符串，包含关键信息
        return f"Vector2D(x={self.x}, y={self.y})"

    def __str__(self):
        # 返回面向用户的友好显示形式
        return f"({self.x}, {self.y})"

# 创建向量实例并观察两种字符串表示
v = Vector2D(3, 4)
print(repr(v))  # 调试友好的输出
print(str(v))   # 面向用户的输出


Vector2D(x=3, y=4)
(3, 4)


### 2. 容器行为：`__len__`、`__iter__`、`__getitem__`

实现这些方法后，自定义类就能与 `len()`、索引操作和迭代协议一起使用，比如在 `for` 循环中遍历。

In [11]:
# 演示实现容器协议的自定义集合类型
class TagCollection:
    def __init__(self, *tags):
        # 将可变参数保存为列表以便后续操作
        self._tags = list(tags)

    def __len__(self):
        # 允许 len() 查询集合规模
        return len(self._tags)

    def __iter__(self):
        # 返回迭代器以支持 for 循环遍历
        return iter(self._tags)

    def __getitem__(self, index):
        # 支持索引访问单个元素
        return self._tags[index]

# 准备示例集合并演示常见用法
tags = TagCollection("python", "魔术方法", "教程")
print(len(tags))         # 触发 __len__
print(tags[0])           # 触发 __getitem__
print(list(tags))        # 触发 __iter__


3
python
['python', '魔术方法', '教程']


### 3. 数值运算与比较：`__add__`、`__lt__` 等

魔术方法可以让自定义类参与算术运算与大小比较，从而自然地融入内置运算符语法。

In [12]:
# 演示算术运算与比较运算的魔术方法
class Budget:
    def __init__(self, amount):
        # 以整数或浮点数表示的预算金额
        self.amount = amount

    def __add__(self, other):
        # 只允许两个 Budget 对象相加
        if not isinstance(other, Budget):
            return NotImplemented
        # 返回新的 Budget 实例，保持不可变语义
        return Budget(self.amount + other.amount)

    def __lt__(self, other):
        # 仅与另一个 Budget 对象比较大小
        if not isinstance(other, Budget):
            return NotImplemented
        # 返回布尔值指示预算是否更小
        return self.amount < other.amount

    def __repr__(self):
        # 提供可读的调试输出
        return f"Budget({self.amount})"

# 准备两个部门预算并演示运算
marketing = Budget(1500)
development = Budget(2000)
total = marketing + development
print(total)                   # 调用 __repr__ 展示结果
print(marketing < development) # 调用 __lt__ 进行比较
print(marketing < 500)         # 非 Budget 比较触发 NotImplemented


Budget(3500)
True


TypeError: '<' not supported between instances of 'Budget' and 'int'

### 4. 上下文管理器：`__enter__` 与 `__exit__`

通过实现这两个方法，可以让对象在 `with` 语句中使用，常见场景包括资源管理、计时器、事务控制等。

In [13]:
# 使用上下文管理器自动跟踪代码块耗时
from datetime import datetime

class Timer:
    def __enter__(self):
        # 记录进入上下文的时间点
        self.start = datetime.now()
        print("开始计时...")
        # 将自身返回给 as 绑定，方便访问
        return self

    def __exit__(self, exc_type, exc, tb):
        # 计算执行耗时并打印结果
        delta = datetime.now() - self.start
        print(f"耗时：{delta.total_seconds():.4f} 秒")
        # 返回 False 表示如有异常继续向外传播
        return False

# 在 with 语句中使用 Timer 自动测量代码段性能
with Timer():
    total = sum(range(100_000))
print(total)  # 打印计算结果以确认逻辑正确


开始计时...
耗时：0.0007 秒
4999950000


### 5. 函数式接口：`__call__`

`__call__` 让对象表现得像函数一样被调用，适用于可配置的算法或需要状态的回调。

In [14]:
# 通过实现 __call__ 让对象表现得像函数
class LinearFunction:
    def __init__(self, slope, intercept):
        # 保存一次函数的斜率与截距
        self.slope = slope
        self.intercept = intercept

    def __call__(self, x):
        # 根据当前配置计算函数值
        return self.slope * x + self.intercept

# 创建一次函数 f(x) = 2x + 1 并像调用函数一样使用
f = LinearFunction(2, 1)
print(f(3))   # 输入 3，返回 7
print(f(10))  # 输入 10，返回 21


7
21
